# Unsupervised Clustering using Hidden Markov Model 

## Import packages

In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from hmmlearn import hmm
from simpl_eeg import (
    connectivity,
    eeg_objects,
    raw_voltage,
    topomap_2d,
    topomap_3d_brain,
    topomap_3d_head,
)
import random

In [ ]:
%matplotlib inline

## Why Hidden Markov Model

A Markov model assumes that the future is conditionally independent of the past given the present (Daniel & James, 2020), with the probability shown below:
$$P(S_i |S_1...S_{i−1}) = P(S_i|S_{i−1})$$ where $S_i$ is the state at time i.

A hidden Markov model (HMM) relates a sequence of observations to a sequence of hidden states that explain the observations (Daniel & James, 2020). For the EEG data, the sequence of observations is the EEG data per time frame and the sequence of hidden states would be the brain states in the dataset. Since the brain activities at time $i$ is less likely to highly correlate to brain activities before time $i-1$, the Markov model assumption would be satisfied at this case and therefore we would like to try to apply the hidden Markov model to EEG data.

Since we don't have labeled data or pre-defined brain states, we would need to use unsupervised HMM for this task. The process of finding the sequence of hidden states given the sequence of observations using HMM is called decoding (Daniel & James, 2020) and the `Viterbi` algorithm is commonly used for decoding. Therefore, for this notebook, I would use the `Viterbi` algorithm in the HMM model for finding the potential brain states. 

A hidden Markov model consists of 5 components:
- the state space: a set of hidden states
- the sequence of observation
- the transition probability matrix: the probability transitioning from state $i$ to state $j$
- the emission probabilities: conditional probabilities for all observations given a hidden state
- the initial probability over states: the probability for the Markov model starts at state $i$

The goal for this task is to explore the set of hidden states (the state space) and the transition probability matrix of the EEG data using hidden Markov model. 

## Read in the Data

In [ ]:
raw_full = mne.io.read_raw_eeglab("../../data/927/fixica.set")

In [ ]:
entire_df = raw_full.to_data_frame()

In [ ]:
channel_names = raw_full.ch_names

In [ ]:
epoch = eeg_objects.Epochs("../../data/927").epoch

## EDA

According to the EDA in the [intro page](https://ubc-mds.github.io/simpl_eeg_capstone/clustering.html), the EEG data is following Gaussian distribution for each electrode node, therefore, a GaussianHMM model would be used. However, there are clearly some outliers in the data. My next step is to remove the outliers of the data based on the expertise suggestions from the partner to keep EEG data which falls into (-50, 50) range.

## Data preprocessing

In [ ]:
# drop rows where all values are zero
cleaned_df = entire_df.loc[(entire_df[channel_names] != 0).all(axis=1)]

# drop the outliers of dataset (only keep rows where EEG voltage is between -50 to 50)
df_no_outliers = cleaned_df.loc[
    ((cleaned_df[channel_names] <= 50) & (cleaned_df[channel_names] >= -50)).all(axis=1)
]

# chunk the data into per second (each second has 2048 readings (rows))
df_second = np.split(df_no_outliers, range(2048, len(df_no_outliers), 2048))


# for each second, randomly sampled 10 time stamps (the original dataset is too big, wants to sample a smaller dataset for exploration)
time_jump = 50
df_second_resample = {}
for second in range(len(df_second)):
    df_second_resample[second] = (
        df_second[second]
        .sample(time_jump, random_state=2020, axis=0)
        .sort_values(by="time")
    )
df_resampled = pd.concat([values for key, values in df_second_resample.items()])

## Building Models

Based on the suggestion from the partner, we would like to explore the data in per 5 second interval. 

In [ ]:
five_second_df = np.split(
    df_resampled, range(time_jump * 5, len(df_resampled), time_jump * 5)
)

In [ ]:
# since HMM model only takes in (n_sample, n_feature) array, reshape the data into an array where each sample has 5 seconds data
chunked_list = []
for i in range(len(five_second_df)):
    chunked_list.append(np.array(five_second_df[i].iloc[:, 1:]).flatten())
chunked_array = np.array(chunked_list)

As mentioned above, the EEG data follows Gaussian distribution and is continous, we would use the `GaussianHMM` model from `hmmlearn` package. Since we don't know the number of brain states from the model, we would like to start with some random numbers.

In [ ]:
# n_components is the number of hidden states (number of brain states)
chunked_model = hmm.GaussianHMM(n_components=14)

In [ ]:
chunked_model.fit(chunked_array)

In [ ]:
chunked_result = chunked_model.decode(chunked_array, algorithm="viterbi")

### Check model output

In [ ]:
# the metric that hmmlearn package itself used to evaluate the model
print(
    f"The log probability for this {chunked_model.n_components}-cluster model is {chunked_result[0]:0.4f}"
)

In [ ]:
print(f"The starting probability for this model is: {chunked_model.startprob_}")

In [ ]:
print(f"The transmission probability matrix of this model is: \n")
pd.DataFrame(chunked_model.transmat_)

#### Interpretation of the output

- Based on the starting probability, the dataset always start with the same brain state with probability equals to 1.
- It is hard to tell whether there are any dominating brain states from the transmission probability. 

In [ ]:
# add lables back to the df
df_result = five_second_df.copy()
for i in range(len(df_result)):
    df_result[i] = df_result[i].assign(cluster=chunked_result[1][i])

In [ ]:
df_result = pd.concat([df_result[i] for i in range(len(df_result))])

In [ ]:
df_result.head()

### Output visualization

#### Reason why I use visualization methods to assess the output

- The log probability that HMM model itself provides doesn't provide contents to evalute the model performance.
- It is hard to evaluate the model performance by only looking at the raw voltage values.
- Visualizing the EEG data seems to be the most intuitive way to check the output.
- There is not a good way to determine the optimal `n_components` (which is the number of brain states in the data) of the HMM model. The best way is to check the output for now.

#### Visualize the average voltage for each cluster in the resampled dataset

In [ ]:
cluster_dict = {}
for i in range(chunked_model.n_components):
    cluster_dict["cluster_" + str(i)] = df_result[df_result["cluster"] == i]

In [ ]:
for cluster_key, cluster in cluster_dict.items():
    cluster_key = topomap_2d.plot_topomap_2d(
        epoch,
        cluster.iloc[:, 1:20].mean().values * 1e-6,
        mark="channel_name",
        cmin=-0.8,
        cmax=0.8,
    )

#### Interpretation

- Each topographic heatmap shows the **average raw voltage** for each electrode node for a specific cluster.
- Although each clusters does show different patterns, it is hard to define the brain states by just looking at it.
- Although there are different patterns in each clusters, the differences are not significant. 

#### Label the original dataset and visualize the outcome

Since we were only drawing a random sample of 50 time stamps for each second for fitting the model, we wanted to add the cluster labels back to the entire dataset to check the outcome to see whether they make a good representation of each second. If the cluster outcomes look very different from the resampled data outcomes, we might want to increase the sample size.

In [ ]:
# add lables back to the entire_df
entire_df_result = np.split(df_no_outliers, range(10240, len(df_no_outliers), 10240))
for i in range(len(entire_df_result)):
    entire_df_result[i] = entire_df_result[i].assign(cluster=chunked_result[1][i])

entire_df_result = pd.concat(
    [entire_df_result[i] for i in range(len(entire_df_result))]
)

In [ ]:
# separate the clusters for the entire dataset
entire_cluster_dict = {}
for i in range(chunked_model.n_components):
    entire_cluster_dict["cluster_" + str(i)] = entire_df_result[
        entire_df_result["cluster"] == i
    ]

In [ ]:
for cluster_key, cluster in entire_cluster_dict.items():
    cluster_key = topomap_2d.plot_topomap_2d(
        epoch,
        cluster.iloc[:, 1:20].mean().values * 1e-6,
        mark="channel_name",
        cmin=-0.5,
        cmax=0.5,
    )

#### Interpretation

- Although the colors don't exactly match with the resampled data, the patterns seem to be consistent. 50 randomly sampled time stamps data seems to serve as a good representation for the entire second.

#### Visualize the raw voltage plot for a couple seconds

Since just assessing the outcome using the topographic heatmap doesn't provide a lot of values, I would like to generate a raw voltage plot to visualize the output. However, since the `raw_voltage` function of the `simpl_eeg` package doesn't accept data frame and don't have access to overwrite the epoch data to include the cluster label, I will use the line plot from `plotly` package. Since the entire dataset is too large, only looking at a screenshot of a five second time period for each clusters. 

In [ ]:
color_list = [
    "red",
    "yellow",
    "green",
    "blue",
    "black",
    "brown",
    "grey",
    "chocolate",
    "crimson",
    "coral",
    "darkgoldenrod",
    "orange",
    "purple",
    "burlywood",
    "cornflowerblue",
    "darkblue",
    "darkviolet",
    "darkorange",
    "darkgray",
]

In [ ]:
color_count = 0
for cluster, df in entire_cluster_dict.items():
    sliced_df = df.iloc[:10240]
    single_plot_df = sliced_df.copy()
    for i, col in enumerate(single_plot_df.columns):
        if col != "cluster":
            single_plot_df[col] = single_plot_df[col] + 100 * i
    single_plot_df["time"] = single_plot_df.index
    single_plot_df = single_plot_df.melt(id_vars=["cluster", "time"])
    line_fig = go.Figure()
    line_fig.add_trace(
        go.Scatter(
            x=single_plot_df["time"],
            y=single_plot_df["value"],
            mode="lines",
            name=cluster,
            line=dict(color=color_list[color_count]),
        )
    )
    line_fig.update_layout(title=cluster)
    line_fig.update_xaxes(showticklabels=False)
    line_fig.update_yaxes(title="Channel", showticklabels=False)
    line_fig.show()
    color_count += 1

#### Interpretation

- **The time interval for each of the clusters above is 5 seconds.**
- There are clearly differences in each of clusters when looking at the raw voltage plot. However, it is hard to interpret whether these patterns are providing any specific information without EEG background knowledge.
- Within the same cluster, the brain activities show similar patterns across different electrode nodes (not exactly the same but similar). 

## Comments about the Hidden Markov Model

- It is really hard to interpret the model and to tune the hyperparameter (`n_components`) for the model.
- Model performance is hard to assess and requires a lot of background knowledge.

## Next Steps

Due to the limited time and efforts that we could allocate to this task, there are other potential useful approaches to try for this task but haven't been implemented yet. 

- Data preprocessing: 
    - instead of sampling only 50 time stamps per second, increase the sampling rate so that it could capture more dynmaics from each second to provide a more accurate result.
    - instead of looking at the entire dataset, subset the dataset into epoches and then use epoched data to fit the model.


- Feature engineering: instead of only using the raw voltage data for model input, include some engineered features that could provide a better representation of the temporal dependencies of the data such as the following:
    - apply rolling mean for each 5 second data chunks rather than simply taking the mean of each 5 second data chunks
    - use the sliding window approach to slide the per 5 second data 


- Literature review: read through more literature articles to define a better metric to evaluate the model

- Hyperparameter tuning: currently, there isn't a better way to find the optimal `# of cluster` in the model other than finish fitting the model and visualizing the output to check. Use the metric that we could locate from the previous objective to tune the hyperparameter.

## Attribution

- Speech and Language Processing. Daniel Jurafsky & James H. Martin. Copyright © 2020. All
rights reserved. Draft of December 30, 2020.
